In [2]:
# setting the environment
## import necessary library and packages
import websocket
import json
import polygon
import sqlite3
import requests
import datetime
from datetime import datetime
import time
import time
import csv
import requests
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from pymongo import MongoClient

In [3]:
#MongoDB Setup
client = MongoClient('mongodb://localhost:27017/')
auxiliary = client['auxiliary']
mongo_fx_data_collection = auxiliary['auxiliary_data_USDEUR']

final_db = client['final_data']
mongo_fx_data_collection_final = final_db['final_data_USDEUR']

In [4]:
#Set up SQLite connection
BaseAuxiliary = declarative_base()
BaseFinal = declarative_base()

# Define classes for Auxiliary DB
class AuxiliaryData(BaseAuxiliary):
    __tablename__ = 'auxiliary_data_USDEUR'
    id = Column(Integer, primary_key=True)
    from_currency = Column(String)
    to_currency = Column(String)
    rate = Column(Float)
    data_timestamp = Column(String)
    rate_timestamp = Column(DateTime)
    max_rate = Column(Float)
    min_rate = Column(Float)
    mean_rate = Column(Float)
    vol = Column(Float)
    fd = Column(Float)

# create engine for Auxiliary DB
engine_auxiliary = create_engine('sqlite:///auxiliary_USDEUR.db')

# use engine_auxiliary to create table
BaseAuxiliary.metadata.create_all(engine_auxiliary)

#Define classes for Final DB
class FinalData(BaseFinal):
    __tablename__ = 'final_data_USDEUR'
    id = Column(Integer, primary_key=True)
    data_timestamp = Column(String)
    db_entry_timestamp = Column(String)
    max_rate = Column(Float)
    min_rate = Column(Float)
    mean_rate = Column(Float)
    vol = Column(Float)
    fd = Column(Float)

# create engine for Auxiliary DB
engine_final = create_engine('sqlite:///final_data_USDEUR.db')

# Use engine_final to create table
BaseFinal.metadata.create_all(engine_final)


/var/folders/yh/cq0k6g5s67vdhvshzm13h_gw0000gn/T/ipykernel_21883/700060366.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  BaseAuxiliary = declarative_base()
/var/folders/yh/cq0k6g5s67vdhvshzm13h_gw0000gn/T/ipykernel_21883/700060366.py:3: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  BaseFinal = declarative_base()


In [5]:
#Define the function to get the data from Polygon API
API_KEY = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'  
def get_real_time_fx_data(currency_from, currency_to, amount, precision):
    try:
        endpoint = f"https://api.polygon.io/v1/conversion/{currency_from}/{currency_to}?amount={amount}&precision={precision}&apiKey={API_KEY}"
        response = requests.get(endpoint)
        response.raise_for_status()  # Raises exception for HTTP errors
        data = response.json()

        if data['status'] == 'success':
            return {
                'data_timestamp': data['last']['timestamp'],
                'rate_timestamp' : datetime.now() ,  # Time of the request
                'rate': data['converted']
                # entry_timestamp will be set just before data entry
            }
        else:
            print("The conversion request was not successful.")
            return None
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [6]:
#How to calculate FD
def calculate_fd(N, max_rate, min_rate):
    # Calculate FD using the formula
    different = max_rate - min_rate
    if different == 0:
        different = 1
    fd = N / different 

    return fd

In [7]:
def calculate_keltner_upper_bands(mean_value, vol):
    return [mean_value + n * 0.025 * vol for n in range(1, 101)]

In [8]:
def calculate_keltner_lower_bands(mean_value, vol):
    return [mean_value - n * 0.025 * vol for n in range(1, 101)]

In [9]:
#Main
def main():
    #Sqlite DB, AUX and FINAL
    Session = sessionmaker(bind=engine_auxiliary)
    session = Session()
    SessionFinal = sessionmaker(bind=engine_final)
    session_final = SessionFinal()

    #Set end time to 6 mins from now
    start_time = datetime.now()
    end_time = start_time + timedelta(seconds = 360)

    #Initial settings for feature
    max_rate = 0
    min_rate = float('inf')
    total_sum = 0
    count = 0
    fd = 0
    pre_vol = 1

#First Loop
    while datetime.now() < end_time:
        rate = get_real_time_fx_data('USD', 'EUR', 1, 4)
        current_time = datetime.now()
        print(rate)
        
        time.sleep(1)

        #update max, min, total_sum, count
        current_rate = rate['rate']
        if max_rate is None or current_rate > max_rate:
            max_rate = current_rate
        if min_rate is None or current_rate < min_rate:
            min_rate = current_rate

        total_sum += current_rate
        count += 1
        
        # Calculate mean and VOL
        mean_value = total_sum / count if count else 0
        vol = (max_rate - min_rate) / mean_value if mean_value else float('inf')

        #calculate mean and VOL
        fx_data = AuxiliaryData(
            data_timestamp = rate['data_timestamp'],
            rate_timestamp = datetime.now(),
            rate = current_rate,
            max_rate = max_rate,
            min_rate = min_rate,
            mean_rate = mean_value,
            vol = vol 
        )

        #Store data in MongoDB
        mongo_fx_data_collection.insert_one({
            'data_timestamp': rate['data_timestamp'],
            'timestamp': datetime.now(),
            'rate': rate,
            'max_rate': max_rate,
            'min_rate': min_rate,
            'mean_value': mean_value,
            'vol': vol
        })  
        session.add(fx_data)
        session.commit()

        # Store data in MongoDB
        mongo_fx_data_collection.insert_one({
            'data_timestamp': rate['data_timestamp'],
            'timestamp': datetime.now(),
            'rate': rate,
            'max_rate': max_rate,
            'min_rate': min_rate,
            'mean_value': mean_value,
            'vol': vol
        })

    #SQLite Record the time data entry into the FINAL DB
    entry_time = datetime.now()
    #Store the first loop data
    final_data = FinalData(
        data_timestamp = rate['data_timestamp'],
        db_entry_timestamp = entry_time,
        max_rate = max_rate,
        min_rate = min_rate,
        mean_rate = mean_value,
        vol = vol,
    )

    session_final.add(final_data)
    session_final.commit()

    #MongoDB Record the time data entry into the FINAL DB
    mongo_fx_data_collection_final.insert_one({
        'data_timestamp': start_time.isoformat(),
        'entry_timestamp': entry_time.isoformat(),
        'max_rate': max_rate,
        'min_rate': min_rate,
        'mean_rate': mean_value,
        'vol': vol
    })

    BaseAuxiliary.metadata.drop_all(engine_auxiliary)
    #Kill the Auxilary DB
    BaseAuxiliary.metadata.create_all(engine_auxiliary)
    mongo_fx_data_collection.delete_many({})

    #set vol to pre_vol for the calculation of next FD
    pre_vol = vol

    #End of the first loop

    print("===================End of first loop ===========================")
    print("max_rate:", max_rate)
    print("min_rate:", min_rate)
    print("mean_value:", mean_value)
    print("vol:", vol)

    #code for #2 to #50
    num_loop = 50
    for i in range(num_loop):
        #SQLite DB
        Session = sessionmaker(bind=engine_auxiliary)
        session = Session()
        SessionFinal = sessionmaker(bind=engine_final)
        session_final = SessionFinal()

        #run 6 mins
        start_time = datetime.now()
        end_time = start_time + timedelta(seconds = 360)

        #Initial settings for feature
        max_rate = 0
        min_rate = float('inf')
        total_sum = 0
        count = 0
        fd = 0
        pre_rate = 0
        N = 0
        #no vol since pre_vol is set in the first loop

        #Start of the loop
        while datetime.now() < end_time:
            rate = get_real_time_fx_data('USD', 'EUR', 1, 4)
            current_time = datetime.now()
            print(rate)

            time.sleep(1)

            #update max, min, total_sum, count
            current_rate = rate['rate']
            if max_rate is None or current_rate > max_rate:
                max_rate = current_rate
            if min_rate is None or current_rate < min_rate:
                min_rate = current_rate
            
            total_sum += current_rate
            count += 1
            
            #calculate FD
            fx_data = AuxiliaryData(
                data_timestamp = rate['data_timestamp'],
                rate_timestamp = datetime.now(),
                rate = current_rate,
                max_rate = max_rate,
                min_rate = min_rate,
                mean_rate = mean_value,
                vol = vol,
                fd = fd
            )

            #store data in MongoDB
            mongo_fx_data_collection.insert_one({
                'data_timestamp' : rate['data_timestamp'],
                'timestamp': datetime.now(),
                'rate': rate,
                'max_rate': max_rate,
                'min_rate': min_rate,
                'mean_value': mean_value,
                'vol': vol,
                'fd': fd
            })
        
        

        #calculate mean and VOL
        mean_value = total_sum / count if count else 0
        vol = (max_rate - min_rate) / mean_value if mean_value else float('inf')
                
        if i > 0:  # Calculate Keltner Upper Bands after the first period
            keltner_upper_bands = calculate_keltner_upper_bands(mean_value, pre_vol)
            print(f"Keltner Upper Bands for period {i}:")
            for n, upper_band in enumerate(keltner_upper_bands, start=1):
                print(f"n={n}, Upper Band={upper_band}")

        if i > 0:  # Calculate Keltner Upper Bands after the first period
            keltner_lower_bands = calculate_keltner_lower_bands(mean_value, pre_vol)
            print(f"Keltner Lower Bands for period {i}:")
            for n, lower_band in enumerate(keltner_upper_bands, start=1):
                print(f"n={n}, Lower Band={lower_band}")

        if vol == 0:
            vol = 1
        #trick to calculate N
        #N += abs(current_rate - pre_rate)/(0.025 * pre_vol + 1e-8)
        #Not sure, but I think N should be represented as 
        N += (current_rate - pre_rate)/(0.025)
        #set current rate to pre_rate for the next calculation
        pre_rate = current_rate
        
        fd = calculate_fd(N, max_rate, min_rate)

        #SQLite:time the data entry for the FINAL DB
        entry_time = datetime.now()

        final_data = FinalData(
            data_timestamp = str(start_time),
            db_entry_timestamp = str(entry_time),
            max_rate = max_rate,
            min_rate = min_rate,
            mean_rate = mean_value,
            vol = vol,
            fd = fd
        )
        session_final.add(final_data)
        session_final.commit()

        #MongoDB:time the data entry for the FINAL DB
        
        mongo_fx_data_collection_final.insert_one({
            'data_timestamp': start_time.isoformat(),
            'entry_timestamp': entry_time.isoformat(),
            'max_rate': max_rate,
            'min_rate': min_rate,
            'mean_rate': mean_value,
            'vol': vol,
            'fd': fd
        })

        print("Data has been saved to the Final DB (SQLite and MongoDB)")

        #reset the tables
        BaseAuxiliary.metadata.drop_all(engine_auxiliary)
        BaseAuxiliary.metadata.create_all(engine_auxiliary)
        mongo_fx_data_collection.delete_many({})

        #set vol to pre_vol for the calculation of next FD
        pre_vol = vol

        print("===================End of loop ===========================")
        print("Number of Loops:", i+1)
        print("Final fd:", fd)

#Convert the data to CSV
engine_final = create_engine('sqlite:///final_data_USDEUR.db')
Session = sessionmaker(bind=engine_final)
session = Session()

#find all the data
records_data = session.query(FinalData).all()

with open('final_data_USDEUR.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'data_timestamp', 'db_entry_timestamp', 'max_rate', 'min_rate', 'mean_rate', 'vol', 'fd']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for record in records_data:
        writer.writerow({
            'id': record.id,
            'data_timestamp': record.data_timestamp,
            'db_entry_timestamp': record.db_entry_timestamp,
            'max_rate': record.max_rate,
            'min_rate': record.min_rate,
            'mean_rate': record.mean_rate,
            'vol': record.vol,
            'fd': record.fd
        })

session.close()
                    




In [10]:
if __name__ == '__main__':
    main()

{'data_timestamp': 1717461992000, 'rate_timestamp': datetime.datetime(2024, 6, 3, 20, 46, 34, 52848), 'rate': 0.9161}
{'data_timestamp': 1717461992000, 'rate_timestamp': datetime.datetime(2024, 6, 3, 20, 46, 35, 323736), 'rate': 0.9161}
{'data_timestamp': 1717461992000, 'rate_timestamp': datetime.datetime(2024, 6, 3, 20, 46, 36, 426929), 'rate': 0.9161}
{'data_timestamp': 1717461992000, 'rate_timestamp': datetime.datetime(2024, 6, 3, 20, 46, 37, 532899), 'rate': 0.9161}
{'data_timestamp': 1717461998000, 'rate_timestamp': datetime.datetime(2024, 6, 3, 20, 46, 38, 626726), 'rate': 0.9161}
{'data_timestamp': 1717461998000, 'rate_timestamp': datetime.datetime(2024, 6, 3, 20, 46, 39, 705047), 'rate': 0.9161}
{'data_timestamp': 1717461998000, 'rate_timestamp': datetime.datetime(2024, 6, 3, 20, 46, 40, 818359), 'rate': 0.9161}
{'data_timestamp': 1717461998000, 'rate_timestamp': datetime.datetime(2024, 6, 3, 20, 46, 41, 912740), 'rate': 0.9161}
{'data_timestamp': 1717461998000, 'rate_timestamp

In [11]:
#find all the data
records_data = session.query(FinalData).all()

with open('final_data_USDEUR.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'data_timestamp', 'db_entry_timestamp', 'max_rate', 'min_rate', 'mean_rate', 'vol', 'fd']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for record in records_data:
        writer.writerow({
            'id': record.id,
            'data_timestamp': record.data_timestamp,
            'db_entry_timestamp': record.db_entry_timestamp,
            'max_rate': record.max_rate,
            'min_rate': record.min_rate,
            'mean_rate': record.mean_rate,
            'vol': record.vol,
            'fd': record.fd
        })

session.close()
                    